# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [18]:
import numpy as np
import pandas as pd
import requests
from scipy import stats
import math
import xlsxwriter


## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [4]:
stocks = pd.read_csv('sp_500_stocks.csv')
IEX_CLOUD_API_TOKEN = 'Tpk_059b97af715d417d9f49f50b51b1c448'


## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [5]:
#rather than pulling price and market Cap, we're looking for Return Metrics
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats/?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data


{'companyName': 'Apple Inc',
 'marketcap': 2259045765840,
 'week52high': 150.73,
 'week52low': 68.93,
 'week52highSplitAdjustOnly': 146.03,
 'week52lowSplitAdjustOnly': 66.5,
 'week52change': 0.944343651490863,
 'sharesOutstanding': 17605377487,
 'float': 0,
 'avg10Volume': 95792301,
 'avg30Volume': 100167292,
 'day200MovingAvg': 129.24,
 'day50MovingAvg': 128.36,
 'employees': 150598,
 'ttmEPS': 3.85,
 'ttmDividendRate': 0.8546087394414587,
 'dividendYield': 0.006103427183236297,
 'nextDividendDate': '0',
 'exDividendDate': '2021-01-27',
 'nextEarningsDate': '2021-04-22',
 'peRatio': 36.25621215943651,
 'beta': 1.351474993493607,
 'maxChangePercent': 52.82211845974128,
 'year5ChangePercent': 4.391067291529741,
 'year2ChangePercent': 1.782590579225062,
 'year1ChangePercent': 0.9153209125720059,
 'ytdChangePercent': 0.01278814633225814,
 'month6ChangePercent': 0.13221271320289016,
 'month3ChangePercent': 0.05863456971687561,
 'month1ChangePercent': 0.07008821283825131,
 'day30ChangePerc

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [6]:
data['year1ChangePercent']

0.9153209125720059

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [7]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

for symbol_string in symbol_strings:
    print(symbol_string)
    
my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,ADI,ADM,ADP,ADSK,AEE,AEP,AES,AFL,AIG,AIV,AIZ,AJG,AKAM,ALB,ALGN,ALK,ALL,ALLE,ALXN,AMAT,AMCR,AMD,AME,AMGN,AMP,AMT,AMZN,ANET,ANSS,ANTM,AON,AOS,APA,APD,APH,APTV,ARE,ATO,ATVI,AVB,AVGO,AVY,AWK,AXP,AZO,BA,BAC,BAX,BBY,BDX,BEN,BF.B,BIIB,BIO,BK,BKNG,BKR,BLK,BLL,BMY,BR,BRK.B,BSX,BWA,BXP,C,CAG,CAH,CARR,CAT,CB,CBOE,CBRE,CCI,CCL,CDNS,CDW,CE,CERN,CF,CFG,CHD,CHRW,CHTR,CI,CINF,CL,CLX,CMA,CMCSA
CME,CMG,CMI,CMS,CNC,CNP,COF,COG,COO,COP,COST,COTY,CPB,CPRT,CRM,CSCO,CSX,CTAS,CTL,CTSH,CTVA,CTXS,CVS,CVX,CXO,D,DAL,DD,DE,DFS,DG,DGX,DHI,DHR,DIS,DISCA,DISCK,DISH,DLR,DLTR,DOV,DOW,DPZ,DRE,DRI,DTE,DUK,DVA,DVN,DXC,DXCM,EA,EBAY,ECL,ED,EFX,EIX,EL,EMN,EMR,EOG,EQIX,EQR,ES,ESS,ETFC,ETN,ETR,EVRG,EW,EXC,EXPD,EXPE,EXR,F,FANG,FAST,FB,FBHS,FCX,FDX,FE,FFIV,FIS,FISV,FITB,FLIR,FLS,FLT,FMC,FOX,FOXA,FRC,FRT,FTI,FTNT,FTV,GD,GE,GILD
GIS,GL,GLW,GM,GOOG,GOOGL,GPC,GPN,GPS,GRMN,GS,GWW,HAL,HAS,HBAN,HBI,HCA,HD,HES,HFC,HIG,HII,HLT,HOLX,HON,HPE,HPQ,HRB,HRL,HSIC,HST,HSY,HUM,HWM,IBM,ICE,IDXX,IEX,IFF,ILM

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [8]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   data[symbol]['stats']['year1ChangePercent'],
                                                   'N/A'
                                                   ], 
                                                  index = my_columns), 
                                        ignore_index = True)
        
    
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,139.880,0.745914,N/A
1,AAL,22.120,1.02695,N/A
2,AAP,203.440,0.658674,N/A
3,AAPL,140.310,0.930202,N/A
4,ABBV,110.560,0.402931,N/A
...,...,...,...,...
500,YUM,119.097,0.513986,N/A
501,ZBH,173.370,0.595562,N/A
502,ZBRA,519.720,1.59555,N/A
503,ZION,58.019,1.19302,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [9]:
final_dataframe.sort_values('One-Year Price Return', ascending = False, inplace = True)
#inplace = true is needed to alter the dataframe
final_dataframe = final_dataframe[:50] #only top 50
final_dataframe.reset_index(inplace=True)
final_dataframe

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,275,LB,67.050,4.50754,N/A
1,179,FCX,38.150,4.18975,N/A
2,208,GPS,33.910,3.38271,N/A
3,272,KSS,59.810,2.46943,N/A
4,78,CARR,44.801,2.46397,N/A
5,24,ALGN,602.235,2.44857,N/A
6,129,DFS,104.920,2.41045,N/A
7,462,URI,323.740,2.33486,N/A
8,253,IVZ,27.393,2.31195,N/A
9,441,TPR,45.604,2.27306,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [10]:
def sizeInput():
    global portfolio_size
    portfolio_size = input('Enter $ value of your portfolio: ')
    try:
        val = float(portfolio_size)
        print(val)
    except ValueError:
        print('you didn\'t put an integer \nPlease try again')
    
sizeInput()

Enter $ value of your portfolio: 10000000
10000000.0


In [11]:
position_size = float(portfolio_size)/len(final_dataframe.index)
for i in range(0, len(final_dataframe)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[i, 'Price'])
    
final_dataframe


C:\Users\robzh\anaconda3\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,275,LB,67.050,4.50754,2982
1,179,FCX,38.150,4.18975,5242
2,208,GPS,33.910,3.38271,5897
3,272,KSS,59.810,2.46943,3343
4,78,CARR,44.801,2.46397,4464
5,24,ALGN,602.235,2.44857,332
6,129,DFS,104.920,2.41045,1906
7,462,URI,323.740,2.33486,617
8,253,IVZ,27.393,2.31195,7301
9,441,TPR,45.604,2.27306,4385


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [28]:
hqm_columns =[
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'One-Year Price Return',
    'One-Year Return Perentile',
    'Six-Month Price Return',
    'Six-Month Return Perentile',
    'Three-Month Price Return',
    'Three-Month Return Perentile',
    'One-Month Price Return',
    'One-Month Return Perentile'
]

hqm_dataframe = pd.DataFrame(columns = hqm_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        hqm_dataframe = hqm_dataframe.append(
            pd.Series(
            [
                symbol,
                data[symbol]['quote']['latestPrice'],
                'N/A',
                data[symbol]['stats']['year1ChangePercent'],
                'N/A',
                data[symbol]['stats']['month6ChangePercent'],
                'N/A',
                data[symbol]['stats']['month3ChangePercent'],
                'N/A',
                data[symbol]['stats']['month1ChangePercent'],
                'N/A',
            ],
            index = hqm_columns),
            ignore_index = True
        )
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Perentile,Six-Month Price Return,Six-Month Return Perentile,Three-Month Price Return,Three-Month Return Perentile,One-Month Price Return,One-Month Return Perentile
0,A,134.259,N/A,0.723769,N/A,0.26986,N/A,0.0695272,N/A,0.100064,N/A
1,AAL,21.840,N/A,1.00094,N/A,0.768604,N/A,0.417495,N/A,-0.102543,N/A
2,AAP,196.760,N/A,0.68267,N/A,0.253524,N/A,0.169256,N/A,0.0806973,N/A
3,AAPL,141.000,N/A,0.913983,N/A,0.132908,N/A,0.0584872,N/A,0.0703564,N/A
4,ABBV,112.804,N/A,0.396532,N/A,0.289682,N/A,-0.0120026,N/A,-0.0146057,N/A
...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,118.390,N/A,0.526054,N/A,0.21372,N/A,0.115789,N/A,0.0879742,N/A
501,ZBH,172.090,N/A,0.593729,N/A,0.177533,N/A,0.0771746,N/A,0.0760887,N/A
502,ZBRA,513.189,N/A,1.54993,N/A,0.76029,N/A,0.255013,N/A,0.0835923,N/A
503,ZION,56.030,N/A,1.16039,N/A,0.865256,N/A,0.152852,N/A,-0.00497871,N/A


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [29]:
time_periods = [
                'One-Year',
                'Six-Month',
                'Three-Month',
                'One-Month'
                ]

for row in hqm_dataframe.index:
    for time_period in time_periods:
        hqm_dataframe.loc[row, f'{time_period} Return Percentile'] = stats.percentileofscore(hqm_dataframe[f'{time_period} Price Return'], hqm_dataframe.loc[row, f'{time_period} Price Return'])/100

# Print each percentile score to make sure it was calculated properly
for time_period in time_periods:
    print(hqm_dataframe[f'{time_period} Return Percentile'])

#Print the entire DataFrame    
hqm_dataframe

TypeError: '<' not supported between instances of 'NoneType' and 'float'

## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [ ]:
from statistics import mean
for row in hqm_dataframe.index[:1]:
    momentum_percentiles = list()
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row,f'{time_period} Return Percentile']
    hqm_dataframe.loc[row,'HQM Score'] = mean(momentum_percentiles)
hqm_dataframe

## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [ ]:
hqm_dataframe.sort_values("HQM Score", ascending = False, inplace = True)
hqm_dataframe = hqm_dataframe[:50]
hqm_dataframe.reset_index(drop = True, inplace = True)
hqm_dataframe



## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [ ]:
portfolio_input()

In [ ]:
position_size = float(portfolio_size)/len(hqm_dataframe.index)
for i in hqm_dataframe.index:
    hqm_dataframe.loc[i, 'Number of Shares to Buy'] = position_size/hqm_dataframe.loc[i,'Price']
hqm_dataframe



## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [ ]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine = 'xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name = 'Momentum Strategy', index = False)


## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [ ]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [ ]:
column_formats = {
    'A':['Ticker', string_template],
    'B':['Price', dollar_template],
    'C':['Number of Shares to Buy', integer_template],
    'D':['One-Year Price Return', percent_template],
    'E':['One-Year Return Perentile',percent_template],
    'F':['Six-Month Price Return', percent_template],
    'G':['Six-Month Return Perentile',percent_template],
    'H':['Three-Month Price Return',percent_template],
    'I':['Three-Month Return Perentile',percent_template],
    'J':['One-Month Price Return',percent_template],
    'K':['One-Month Return Perentile',percent_template],
    'L':['HQM Score',percent_template]
}

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 25,column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])
    

## Saving Our Excel Output

As before, saving our Excel output is very easy:

In [ ]:
writer.save()